# OC - Parcours Ingénieur IA - Projet 1 - BERKAN Asli Ceren

## Préparation de l'environnement

In [1]:
# pip install dotenv # installation pour la sécurisation des données de connexions
# pip install azure-ai-textanalytics --pre # installation azure

In [12]:
import os # pour récupérer les données dans env
from dotenv import load_dotenv # pour sécuriser la clé, le lien et la location
load_dotenv()

True

In [13]:
subscription_key = os.getenv("TRANSLATOR_KEY")
endpoint = os.getenv("TRANSLATOR_ENDPOINT")
location = os.getenv("TRANSLATOR_RESOURCE_LOCATION")

In [4]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(subscription_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

## Un premier exemple avec détection de la langue 
https://docs.microsoft.com/fr-fr/azure/cognitive-services/text-analytics/quickstarts/client-libraries-rest-api?tabs=version-3-1&pivots=programming-language-python#language-detection

In [5]:
def language_detection_example(client):
    try:
        documents = ["Schiedam is gelegen tussen Rotterdam en Vlaar. "]
        response = client.detect_language(documents = documents, country_hint = 'us')[0]
        print("Language: ", response.primary_language.name)

    except Exception as err:
        print("Encountered exception. {}".format(err))
language_detection_example(client)

Language:  Dutch


## Détection à partir du jeu de données 'Wikipedia Language Identification Database'

Les langues les plus parlés en 2021 selon le site 'https://www.ethnologue.com/' sont : Anglais, Chinois, Hindu, Espagnol et Arabe Standard. 

In [6]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

MostSpokenLanguage = ['en', 'zh', 'hi', 'es', 'ar']

# Lecture des textes et des langues
data = pd.read_csv("Dataset/x_test.txt", delimiter = "\r\n", engine='python', encoding="utf-8", header = None)
data.columns =  ['text']
data['labels'] = pd.read_csv("Dataset/y_test.txt", delimiter = " ", header = None, keep_default_na=False)
data['language']= 'en'
print(data)

                                                     text labels language
0       Ne l fin de l seclo XIX l Japon era inda çconh...    mwl       en
1       Schiedam is gelegen tussen Rotterdam en Vlaard...    nld       en
2       ГIурусаз батальонал, гьоркьор гIарадабиги лъун...    ava       en
3       ರಾಜ್ಯಶಾಸ್ತ್ರದ ಪಿತಾಮಹೆ ಅರಿಸ್ಟಾಟಲ್. ರಾಜ್ಯಶಾಸ್ತ್ರ...    tcy       en
4       Halukum adalah kelenjar tiroid nang menonjol d...    bjn       en
...                                                   ...    ...      ...
117495  Wakati wa mimba,homa ya Q ni vigumu kutibu kwa...    swa       en
117496  گیلون یک ته تاریخی منطقه‌ سفیدرود دلتای طرف ای...    glk       en
117497  តាម​រយៈ​ការ​ចិញ្ចឹម​មនោសញ្ចេតនា​ជាតិនិយម​បែប​ន...    khm       en
117498  روس اک وفاق اے تے 1 مارچ 2008ء توں اسدیاں 83 و...    pnb       en
117499  Chute annaée-lo, la Normaundie n'est pus recou...    nrm       en

[117500 rows x 3 columns]


In [7]:
# Lecture des labels Wiki pour convertir le label langue en ISO (pas obligatoire pour envoi des données à l'API)
labels_Wiki = pd.read_csv("Dataset/labels.csv", sep=';', keep_default_na=False, na_values=[''])
labels_Wiki_Label = labels_Wiki.Label
#print(labels_Wiki_Label)

for i in range(len(data.labels)):
    ind = labels_Wiki.index[labels_Wiki.Label == data.labels[i]].tolist()
    data.language[i] = labels_Wiki.at[ind[0], 'Wiki Code']
print(data) 

                                                     text labels language
0       Ne l fin de l seclo XIX l Japon era inda çconh...    mwl      mwl
1       Schiedam is gelegen tussen Rotterdam en Vlaard...    nld       nl
2       ГIурусаз батальонал, гьоркьор гIарадабиги лъун...    ava       av
3       ರಾಜ್ಯಶಾಸ್ತ್ರದ ಪಿತಾಮಹೆ ಅರಿಸ್ಟಾಟಲ್. ರಾಜ್ಯಶಾಸ್ತ್ರ...    tcy      tcy
4       Halukum adalah kelenjar tiroid nang menonjol d...    bjn      bjn
...                                                   ...    ...      ...
117495  Wakati wa mimba,homa ya Q ni vigumu kutibu kwa...    swa       sw
117496  گیلون یک ته تاریخی منطقه‌ سفیدرود دلتای طرف ای...    glk      glk
117497  តាម​រយៈ​ការ​ចិញ្ចឹម​មនោសញ្ចេតនា​ជាតិនិយម​បែប​ន...    khm       km
117498  روس اک وفاق اے تے 1 مارچ 2008ء توں اسدیاں 83 و...    pnb      pnb
117499  Chute annaée-lo, la Normaundie n'est pus recou...    nrm      nrm

[117500 rows x 3 columns]


In [8]:
# Réduction des textes aux 5 langues les plus parlées
data_MostSpokenLanguage = data[data.language.isin(MostSpokenLanguage)]

# Ajout d'une colonne ID
data_MostSpokenLanguage.insert(0, "id", range(len(data_MostSpokenLanguage)), allow_duplicates=False)

# Supprimer la colonne labels 
data_MostSpokenLanguage = data_MostSpokenLanguage.drop('labels',axis=1)

print(data_MostSpokenLanguage)
#print(data_MostSpokenLanguage.shape)

          id                                               text language
15         0  كانت رحلة كوك - فولسوم - بيترسون الاستكشافية ع...       ar
42         1  大都会区有它自己的当地路边快餐口味，包括瓦达帕夫（蓬松面包劈开一半，填入锅贴）、潘尼普里（油...       zh
50         2  La ciudad de San Cristóbal es sede del Hospita...       es
68         3  Anton (or Antonius) Maria Schyrleus (also Schy...       en
113        4  बांग्लादेश के मुख्य न्यायाधीश का पद, बांग्लादे...       hi
...      ...                                                ...      ...
117200  2495  Fue fundado el 18 de octubre de 1963. El día 2...       es
117278  2496  戈尔巴乔夫结束了苏共的专制和暴政，使人民获得了民主、法治和自由，并使得东欧国家自主发展，结束...       zh
117282  2497  El investigador ha recibido varios reconocimie...       es
117464  2498  Con motivo de la celebración del Septuagésimoq...       es
117471  2499  1999年10月，當時還只有16歲的她在美國出道，以Mai-K名義推出首張英文《Baby I...       zh

[2500 rows x 3 columns]


In [9]:
# Réarrangement et réduction du texte pour envoie
n = 30 
data_MostSpokenLanguage_n = data_MostSpokenLanguage.groupby('language').head(n)
print(data_MostSpokenLanguage_n)
print(data_MostSpokenLanguage_n.shape)

# Convertir la dataframe en doc
test = data_MostSpokenLanguage_n.to_dict('records') # ne marche pas avec to_json()
#print(test)

       id                                               text language
15      0  كانت رحلة كوك - فولسوم - بيترسون الاستكشافية ع...       ar
42      1  大都会区有它自己的当地路边快餐口味，包括瓦达帕夫（蓬松面包劈开一半，填入锅贴）、潘尼普里（油...       zh
50      2  La ciudad de San Cristóbal es sede del Hospita...       es
68      3  Anton (or Antonius) Maria Schyrleus (also Schy...       en
113     4  बांग्लादेश के मुख्य न्यायाधीश का पद, बांग्लादे...       hi
...   ...                                                ...      ...
7616  155  Un capitán robot del CDF, líder de sus tropas....       es
8113  164  हाल के वर्षों में, सामाजिक उद्यम क्षेत्र ने फ़...       hi
8381  169  इस फ़िल्म में पाँच युगलों को ऊटी में अपने हनीम...       hi
8410  171  न्यायमूर्ति मोहम्मद अजमल मियां , पाकिस्तान की ...       hi
8764  176  पहली शताब्दी ईसा से सबूत कुछ यूनानियों के साथ ...       hi

[150 rows x 3 columns]
(150, 3)


In [10]:
# Analyse avec Azure du document
detectionAzure = client.detect_language(documents = test)
#print(detectionAzure)

In [11]:
# Récupération des données

# Ajout d'une colonne detectedLanguageAzure et scores pour le retour Azure
data_MostSpokenLanguage_n.insert(3, "detectedLanguageAzure", range(len(data_MostSpokenLanguage_n)), allow_duplicates=False)
data_MostSpokenLanguage_n.insert(4, "scores", range(len(data_MostSpokenLanguage_n)), allow_duplicates=False)

# Initialisation des listes pour le code iso de la langue et le score
getisoname = ['fr']*len(detectionAzure)
getscore = [0]*len(detectionAzure)

# Boucle de récup
for i in range(len(detectionAzure)):
    getisoname[i] = detectionAzure[i].primary_language.iso6391_name
    getscore[i] = detectionAzure[i].primary_language.confidence_score
    
#print(getisoname)
data_MostSpokenLanguage_n.detectedLanguageAzure = getisoname
data_MostSpokenLanguage_n.scores = getscore
print(data_MostSpokenLanguage_n)

       id                                               text language  \
15      0  كانت رحلة كوك - فولسوم - بيترسون الاستكشافية ع...       ar   
42      1  大都会区有它自己的当地路边快餐口味，包括瓦达帕夫（蓬松面包劈开一半，填入锅贴）、潘尼普里（油...       zh   
50      2  La ciudad de San Cristóbal es sede del Hospita...       es   
68      3  Anton (or Antonius) Maria Schyrleus (also Schy...       en   
113     4  बांग्लादेश के मुख्य न्यायाधीश का पद, बांग्लादे...       hi   
...   ...                                                ...      ...   
7616  155  Un capitán robot del CDF, líder de sus tropas....       es   
8113  164  हाल के वर्षों में, सामाजिक उद्यम क्षेत्र ने फ़...       hi   
8381  169  इस फ़िल्म में पाँच युगलों को ऊटी में अपने हनीम...       hi   
8410  171  न्यायमूर्ति मोहम्मद अजमल मियां , पाकिस्तान की ...       hi   
8764  176  पहली शताब्दी ईसा से सबूत कुछ यूनानियों के साथ ...       hi   

     detectedLanguageAzure  scores  
15                      ar    1.00  
42                  zh_chs    0.99  
50          